In [2]:
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import pandas as pd
import os
import torch
from tqdm import tqdm
from pathlib import Path

Loading dataset

In [ ]:
path = "" # DATASET PATH
IMAGES_PATH = path + '/images'
# Load sales data
train_df = pd.read_csv(Path(path + '/train.csv'), parse_dates=['release_date'])
test_df = pd.read_csv(Path(path + '/test.csv'), parse_dates=['release_date'])

Load Clip

In [4]:
model = CLIPModel.from_pretrained("patrickjohncyh/fashion-clip").to("cuda")
processor = CLIPProcessor.from_pretrained("patrickjohncyh/fashion-clip")

In [6]:
def load_imagePath_2_PIL(img_path):
    return Image.open(f"{IMAGES_PATH}/{img_path}")

In [7]:
embeddings_file = "image_embeddings.pt"

Store embeddings

In [11]:
embeddings = {}
with torch.no_grad():  # Disable gradient calculation for inference
    for img_name in tqdm(train_df['image_path']):
        img_path = f"{IMAGES_PATH}/{img_name}"
        try:
            image = Image.open(img_path)
        except:
            print(f"Error opening image {img_path}")

        # Preprocess the image with the FASHION-CLIP processor
        inputs = processor(images=image, return_tensors="pt", padding=True).to("cuda")
        
        # Get image embeddings
        image_features = model.get_image_features(**inputs)
        image_features = image_features.squeeze().cpu()

        # Store embeddings in a dictionary
        embeddings[img_name] = image_features

# Save embeddings to a file
torch.save(embeddings, embeddings_file)
print(f"Embeddings saved to {embeddings_file}")

100%|██████████| 5080/5080 [01:33<00:00, 54.46it/s]


Embeddings saved to image_embeddings.pt


In [13]:
embeddings_file = "image_embeddings_test.pt"

embeddings_test = {}
with torch.no_grad():  # Disable gradient calculation for inference
    for img_name in tqdm(test_df['image_path']):
        img_path = f"{IMAGES_PATH}/{img_name}"
        try:
            image = Image.open(img_path)
        except:
            print(f"Error opening image {img_path}")

        # Preprocess the image
        inputs = processor(images=image, return_tensors="pt", padding=True).to("cuda")
        
        # Get image embeddings
        image_features = model.get_image_features(**inputs)
        image_features = image_features.squeeze().cpu()  # Convert to a 1D tensor and move to CPU

        # Store embeddings in a dictionary
        embeddings_test[img_name] = image_features

torch.save(embeddings_test, embeddings_file)
print(f"Embeddings saved to {embeddings_file}")

100%|██████████| 497/497 [00:11<00:00, 42.85it/s]

Embeddings saved to image_embeddings_test.pt
